# Formation Normalization

In [2]:
import pandas as pd
import numpy as np
import pickle
import pyodbc
import numba
import os
import sys

sys.path.append("S:\\Engineering\\01. Development\\Financial Engine\\")

from utils.pandas_utils import *
from utils.custom_tfs import *

In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Needs 1002a, formations, allocations, section_assumptions data

In [4]:
# db connections
driver = "{ODBC Driver 17 for SQL Server}"
server = "syntax-dev-sql.database.windows.net"
port = 1443

user = os.environ["ALLSQUARE_USERNAME"]
pwd = os.environ["ALLSQUARE_PASSWORD"]
auth = "ActiveDirectoryPassword"
cnxns = dict()
dbs = ["syntax_dev", "syntax_dev_hist", "stg_cg_export"]
for db in dbs:
    cnxn_string = f"DRIVER={driver};PORT={port};SERVER={server};PORT={port};DATABASE={db};UID={user};PWD={pwd};Authentication={auth}"
    cnxns[db] = pyodbc.connect(cnxn_string)

<IPython.core.display.Javascript object>

In [29]:
get_index_well = """
SELECT
    api
    from dbo.index_well
where is_horizontal = 1
"""

get_w27_main = """
SELECT
    id,
    n_api as api,
    well_name,
    well_number,
    operator_name,
    mod_date,
    well_completion
from r_occ_w27_main
"""

get_w27_formation = """
SELECT
    id,
    n_api as api,
    form_name
from r_occ_w27_formation
where form_name != ''
"""

get_allocation = """
SELECT 
    api,
    trsm_heh,
    allocation,
    proxy_allocation,
    footage,
    footage/proxy_allocation as total_footage
    FROM dbo.index_well_land
where proxy_allocation is not null
    and use_for_allocation = 1; 
"""

<IPython.core.display.Javascript object>

In [30]:
horizontal_wells = (
    pd.read_sql(get_index_well, cnxns["stg_cg_export"])
    .pipe(ccast, (["api"], tf_api))
    .api.unique()
)

<IPython.core.display.Javascript object>

## 1002a

In [31]:
# In case of multiple f1002a for a single api take the latest one.
f1002a = (
    pd.read_sql(get_w27_main, cnxns["stg_cg_export"])
    .pipe(ccast, (["api"], tf_api))
    .pipe(ccast, (["mod_date", "well_completion"], tf_date))
    .assign(
        well_name=lambda x: x.well_name.str.strip()
        + " "
        + x.well_number.str.strip()
    )
    .query("api in @horizontal_wells")
    .reset_index(drop=True)
    .loc[
        :,
        [
            "id",
            "api",
            "well_name",
            "operator_name",
            "mod_date",
            "well_completion",
        ],
    ]
    .sort_values(["api", "mod_date"], ascending=[True, False])
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

### Filter wells that dont have duplicates. In case of duplicates keep on the latest (based on modified date) as the only 1002a for the well.

In [33]:
unique_mask = ~f1002a.duplicated("api", keep="first") | ~f1002a.duplicated(
    "api", keep=False
)

<IPython.core.display.Javascript object>

## Formation

In [34]:
f1002a_id = f1002a.id

<IPython.core.display.Javascript object>

In [35]:
formation = (
    pd.read_sql(get_w27_formation, cnxns["stg_cg_export"])
    .query("id in @f1002a_id")
    .reset_index(drop=True)
    .assign(form_name=lambda x: x.form_name.str.strip())
    .pipe(ccast, (["form_name"], tf_formation))
    .groupby("id", as_index=False)
    .form_name.agg({"formation": lambda x: " ".join(x)})
)

<IPython.core.display.Javascript object>

In [36]:
allocation_threshold = 0.15

<IPython.core.display.Javascript object>

In [37]:
allocation = (
    pd.read_sql(get_allocation, cnxns["stg_cg_export"])
    .pipe(ccast, (["api"], tf_api))
    .pipe(ccast, (["footage", "total_footage"], tf_feet))
    .pipe(ccast, (["allocation", "proxy_allocation"], tf_allocation))
    .assign(
        final_allocation=lambda x: np.where(
            x.allocation.notnull(), x.allocation, x.proxy_allocation
        )
    )
    .rename(columns={"final_allocation": "allocation"})
    .query("allocation > @allocation_threshold")
    .loc[:, ["api", "trsm_heh"]]
)

<IPython.core.display.Javascript object>


## Section Assumption

In [38]:
sec_ass = (
    pd.read_csv("../data/section_assumption.csv")
    .pipe(
        ccast,
        (
            ["nwells_1", "tolerance_1", "nwells_2", "tolerance_2"],
            tf_number_wells,
        ),
    )
    .pipe(ccast, (["section_acres"], tf_section_acres))
    .pipe(ccast, (["formation_1", "formation_2"], tf_formation))
)

<IPython.core.display.Javascript object>

In [39]:
df = (
    f1002a.merge(allocation, on="api")
    .merge(
        sec_ass.query("formation_1 != 'NONE' or formation_2 != 'NONE'").loc[
            :, ["trsm_heh", "formation_1", "formation_2"]
        ],
        on="trsm_heh",
    )
    .reset_index(drop=True)
    .merge(formation, on="id")
)

<IPython.core.display.Javascript object>

In [40]:
f"Processing {df.shape[0]} wells for formation normalization"

'Processing 19754 wells for formation normalization'

<IPython.core.display.Javascript object>

## Logic starts here
### A string array of dimenstion - [5, nwells]
Array should be oneline. One row per well.
0. well_name 
1. operator_name
2. formation (concatenated with space)
3. formation 1 (from section assumption)
4. formation 2 (from section assumption)

In [41]:
string_columns = [
    "well_name",
    "operator_name",
    "formation",
    "formation_1",
    "formation_2",
]
string_array = df.loc[:, string_columns].values.transpose().astype("unicode")
wells = string_array[0]
operators = string_array[1]
formations = string_array[2]
formation_1 = string_array[3]
formation_2 = string_array[4]

known_formations = np.array(
    [
        "WOODFORD",
        "MERAMEC",
        "SPRINGER",
        "SYCAMORE",
        "OSAGE",
        "OSWEGO",
        "HOXBAR",
        "MAYES",
        "DES MOINES",
        "MARMATON",
        "CLEVELAND",
        "COTTAGE GROVE",
        "TONKAWA",
    ]
).astype(string_array.dtype.str)

columns, rows = string_array.shape
use_dtype = string_array.dtype

norm_form = np.chararray((rows), unicode=True).astype(use_dtype)

assert (
    norm_form.dtype == known_formations.dtype == string_array.dtype
), "dtypes not matching"

# If formation is from known formation assign it
norm_form = np.where(
    np.apply_along_axis(lambda x: np.isin(x, known_formations), 0, formations),
    formations,
    norm_form,
)

# If formation is not present and formation 1 exists then that is the formation
norm_form = np.where(
    (norm_form == "") & (formation_1 != ""), formation_1, norm_form
)

# If Operator equals Casillas and Well Name contains "MXH" or "MH", then SYCAMORE.
well_mxh_mh = (np.char.find(wells, "MXH") >= 0) | (
    np.char.find(wells, "MH") >= 0
)
norm_form = np.where(
    (norm_form == "")
    & (np.char.find(operators, "CASILLAS") >= 0)
    & well_mxh_mh,
    "SYCAMORE",
    norm_form,
)

# If Formation 1 or Formation 2 contain either SYCAMORE, SPRINGER, or OSAGE (all Mississippian aged rocks)
# AND the second formation is NOT SYCAMORE, SPRINGER, OR OSAGE
# AND, Well Name contains "MXH" or "MH"
# THEN, normalized to the Miss Formation listed
miss_rock = np.array(["SYCAMORE", "SPRINGER", "OSAGE"], dtype=use_dtype)
form_1_miss = np.apply_along_axis(
    lambda x: np.isin(x, miss_rock), 0, formation_1
)
form_2_miss = np.apply_along_axis(
    lambda x: np.isin(x, miss_rock), 0, formation_2
)

both_form_miss = form_1_miss & form_2_miss
norm_form = np.where(
    (form_1_miss & ~both_form_miss & well_mxh_mh), formation_1, norm_form
)
norm_form = np.where(
    (form_2_miss & ~both_form_miss & well_mxh_mh), formation_2, norm_form
)

# If formation contains 'WOODFORD' assign it WOODFORF
norm_form = np.where(
    (norm_form == "") & (np.char.find(formations, "WOODFORD") >= 0),
    "WOODFORD",
    norm_form,
)

# If Operator contains 'OKLAHOMA ENERGY ACQUISITIONS' make formation 'OSAGE'
norm_form = np.where(
    (norm_form == "")
    & (np.char.find(operators, "OKLAHOMA ENERGY ACQUISITIONS") >= 0),
    "OSAGE",
    norm_form,
)

# If formation 1 or 2 is 'MERAMEC' and formation contains "MISS" then "MERAMEC"
norm_form = np.where(
    (norm_form == "")
    & (
        ((formation_1 == "MERAMEC") | (formation_2 == "MERAMEC"))
        & (np.char.find(formations, "MISS") >= 0)
    ),
    "MERAMEC",
    norm_form,
)

# If formation contains 'SPRINGER' or 'GODDARD' then 'SPRINGER'
norm_form = np.where(
    (norm_form == "")
    & (
        (np.char.find(formations, "SPRINGER") >= 0)
        | (np.char.find(formations, "GODDARD") >= 0)
    ),
    "SPRINGER",
    norm_form,
)

# If formation 1 contains "OSWEGO" then formation 2
norm_form = np.where(
    (norm_form == "") & (formation_1 == "OSWEGO"), formation_2, norm_form
)

# If formation 1 contains "WOODFORD" then formation 2
norm_form = np.where(
    (norm_form == "") & (formation_1 == "WOODFORD"), formation_2, norm_form
)

<IPython.core.display.Javascript object>

In [42]:
f"{(norm_form == '').sum()}/{norm_form.shape[0]} wells have no normalized formations"

'4175/19754 wells have no normalized formations'

<IPython.core.display.Javascript object>

In [43]:
[f"{x} - {(norm_form == x).sum()}" for x in np.unique(norm_form)]

[' - 4175',
 'CLEVELAND - 482',
 'COTTAGE GROVE - 4',
 'DES MOINES - 74',
 'GRANITE WASH - 487',
 'HOXBAR - 99',
 'MARMATON - 276',
 'MERAMEC - 4084',
 'MISS LIME - 2455',
 'OSAGE - 670',
 'OSWEGO - 375',
 'SPRINGER - 1142',
 'SYCAMORE - 94',
 'TONKAWA - 15',
 'WOODFORD - 5322']

<IPython.core.display.Javascript object>

In [44]:
df["norm_form"] = norm_form

<IPython.core.display.Javascript object>

In [45]:
df.loc[
    :,
    [
        "id",
        "api",
        "well_name",
        "trsm_heh",
        "operator_name",
        "well_completion",
        "norm_form",
    ],
].rename(columns={"norm_form": "formation"}).reset_index(drop=True).to_pickle(
    "../data/eff_sec_form_norm.pickle"
)

<IPython.core.display.Javascript object>